In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.nn.functional import one_hot
from matplotlib import pyplot as plt

In [2]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.clf = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=32,kernel_size=6, bias=False), # 28-4+1 = 25
            nn.BatchNorm2d(num_features=32),
            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=5, bias=False), #25-4+1 = 22
            nn.BatchNorm2d(num_features=16),
            nn.Conv2d(in_channels=16,out_channels=8,kernel_size=4, bias=False),
            nn.BatchNorm2d(num_features=8),
            nn.Flatten(),
            nn.Linear(2048,1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024,512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,10),     
        )

    def forward(self, x):
        return self.clf(x)

In [3]:
dataloader = DataLoader(
    MNIST('../Python/', download=False, train=True,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomAffine(degrees=(-10,10),translate=(0.1,0.1))
    ])),
     batch_size=64,
    shuffle=True)

In [4]:
criterion = torch.nn.CrossEntropyLoss()
clf = Classifier()
optim = torch.optim.Adam(clf.parameters())

In [5]:
for e in range(5):
    lossmean = []
    for _, (x,y) in enumerate(dataloader):
        pred = clf(x)

        loss = criterion(pred,one_hot(y,num_classes=10).float() )
        
        optim.zero_grad()
        loss.backward()
        optim.step()

        lossmean.append(loss.item())
    
    print(torch.mean(torch.FloatTensor(lossmean)))


tensor(0.4766)
tensor(0.2776)
tensor(0.2155)
tensor(0.1918)
tensor(0.1676)


In [6]:
test_data =  DataLoader(
    MNIST('../Python/', download=False, train=False,transform=transforms.ToTensor()),
     batch_size=64,
    shuffle=False)

In [7]:
clf.eval()
with torch.no_grad():
    sum = 0
    samples = 0
    for _,(x,y) in enumerate(test_data):
        p = clf(x)
        sum += torch.sum(torch.argmax(p, dim=1) == y).item()
        samples += y.size(0)
        

    print(sum/samples)
clf.train()

0.9715


Classifier(
  (clf): Sequential(
    (0): Conv2d(1, 32, kernel_size=(6, 6), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), bias=False)
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(16, 8, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=2048, out_features=1024, bias=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Linear(in_features=1024, out_features=512, bias=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [8]:
torch.save(clf.state_dict(), 'modelAugmented.pt')